In [ ]:
import sys, os; sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__) if '__file__' in globals() else os.getcwd(), '..')))
#import os; os.chdir(os.path.dirname(os.getcwd()))
from utils.model_loader import get_model_fits
import numpy as np
import pandas as pd
import re
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
data_dir = f"datasets/friedman"
results_dir_relu = "results/regression/single_layer/relu/friedman/no_lambda"
results_dir_tanh = "results/regression/single_layer/tanh/friedman/no_lambda"

model_names_relu_no_lambda = ["Dirichlet tau", "Beta tau", "Dirichlet", "Beta"]
model_names_tanh_no_lambda = ["Dirichlet tau tanh", "Beta tau tanh", "Dirichlet tanh", "Beta tanh"]

relu_fits_no_lambda = {}
tanh_fits_no_lambda = {}

files = sorted(f for f in os.listdir(data_dir) if f.endswith(".npz"))
for fname in files:
    base_config_name = fname.replace(".npz", "")  # e.g., "GAM_N100_p8_sigma1.00_seed1"
    full_config_path = f"{base_config_name}"  # → "type_1/GAM_N100_p8_sigma1.00_seed1"
    relu_fit = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_relu,
        models=model_names_relu_no_lambda,
        include_prior=False,
    )
    
    tanh_fit = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_tanh,
        models=model_names_tanh_no_lambda,
        include_prior=False,
    )
    

    relu_fits_no_lambda[base_config_name] = relu_fit  # use clean key
    tanh_fits_no_lambda[base_config_name] = tanh_fit  # use clean key
    


In [ ]:
data_dir = f"datasets/friedman_correlated"
results_dir_relu_correlated = "results/regression/single_layer/relu/friedman_correlated/no_lambda"
results_dir_tanh_correlated = "results/regression/single_layer/tanh/friedman_correlated/no_lambda"

relu_fits_correlated_no_lambda = {}
tanh_fits_correlated_no_lambda = {}
files = sorted(f for f in os.listdir(data_dir) if f.endswith(".npz"))
for fname in files:
    base_config_name = fname.replace(".npz", "")  # e.g., "GAM_N100_p8_sigma1.00_seed1"
    full_config_path = f"{base_config_name}"  # → "type_1/GAM_N100_p8_sigma1.00_seed1"
    relu_fit_correlated = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_relu_correlated,
        models=model_names_relu_no_lambda,
        include_prior=False,
    )
    
    tanh_fit_correlated = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_tanh_correlated,
        models=model_names_tanh_no_lambda,
        include_prior=False,
    )
    

    relu_fits_correlated_no_lambda[base_config_name] = relu_fit_correlated  # use clean key
    tanh_fits_correlated_no_lambda[base_config_name] = tanh_fit_correlated  # use clean key
    


In [4]:
import re
import numpy as np
import pandas as pd
from properscoring import crps_ensemble
from scores.probability import crps_for_ensemble

_FRIEDMAN_KEY = re.compile(r"Friedman_N(\d+)_p\d+_sigma([\d.]+)_seed(\d+)")

def extract_friedman_metadata(key: str):
    """
    Parse 'Friedman_N{N}_p10_sigma{sigma}_seed{seed}' -> (N:int, sigma:float, seed:int)
    Returns (None, None, None) if it doesn't match.
    """
    m = _FRIEDMAN_KEY.search(key)
    if not m:
        return None, None, None
    N = int(m.group(1))
    sigma = float(m.group(2))
    seed = int(m.group(3))
    return N, sigma, seed


In [5]:
def compute_rmse_from_fits(all_fits, model_names=None, folder="friedman"):
    """
    Iterate over all dataset keys in `all_fits` (e.g., relu_fits or tanh_fits).
    For each model in `model_names` (or all models found if None), compute:
      - RMSE for each posterior draw
      - RMSE of the posterior mean predictor

    Returns:
        df_rmse: long DF with one row per posterior draw.
        df_posterior_rmse: one row per model/dataset with posterior-mean RMSE.
    """
    rmse_rows = []
    post_mean_rows = []

    for dataset_key, model_dict in all_fits.items():
        N, sigma, seed = extract_friedman_metadata(dataset_key)
        if N is None:
            # Skip non-Friedman entries if any
            continue
        
        try:
            path = f"datasets/{folder}/Friedman_N{N}_p10_sigma{sigma:.2f}_seed{seed}.npz"
            data = np.load(path)
            y_test = data["y_test"].squeeze()  # shape (N_test,)
        except FileNotFoundError:
            path = f"datasets/{folder}/many/Friedman_N{N}_p10_sigma{sigma:.2f}_seed{seed}.npz"
            data = np.load(path)
            y_test = data["y_test"].squeeze()  # shape (N_test,)
            #print(f"[SKIP] y_test not found: {path}")
            #continue

        # Choose which models to evaluate
        models_to_eval = model_names or list(model_dict.keys())

        for model in models_to_eval:
            # Some entries may be missing
            entry = model_dict.get(model, None)
            if not entry or "posterior" not in entry:
                print(f"[SKIP] Missing posterior: {dataset_key} -> {model}")
                continue

            fit = entry["posterior"]

            # Expecting (S, N_test, 1) or (S, N_test)
            output_test = fit.stan_variable("output_test")
            if output_test.ndim == 3 and output_test.shape[-1] == 1:
                preds = output_test[..., 0]  # (S, N_test)
            elif output_test.ndim == 2:
                preds = output_test  # (S, N_test)
            else:
                raise ValueError(f"Unexpected output_test shape {output_test.shape} for {dataset_key} -> {model}")

            # Per-sample RMSE
            sq_err = (preds - y_test[None, :])**2  # (S, N_test)
            rmse_per_sample = np.sqrt(np.mean(sq_err, axis=1))  # (S,)

            for s_idx, rmse in enumerate(rmse_per_sample):
                rmse_rows.append({
                    "dataset_key": dataset_key,
                    "model": model,
                    "N": N,
                    "sigma": sigma,
                    "seed": seed,
                    "sample_idx": s_idx,
                    "rmse": float(rmse)
                })

            # Posterior-mean RMSE
            posterior_mean = preds.mean(axis=0)  # (N_test,)
            post_mean_rmse = float(np.sqrt(np.mean((posterior_mean - y_test)**2)))
            post_mean_rows.append({
                "dataset_key": dataset_key,
                "model": model,
                "N": N,
                "sigma": sigma,
                "seed": seed,
                "posterior_mean_rmse": post_mean_rmse
            })

    df_rmse = pd.DataFrame(rmse_rows)
    df_posterior_rmse = pd.DataFrame(post_mean_rows)
    return df_rmse, df_posterior_rmse

In [ ]:
# Evaluate ReLU models
df_rmse_relu_no_lambda, df_posterior_rmse_relu_no_lambda = compute_rmse_from_fits(
    relu_fits_no_lambda, model_names_relu_no_lambda  # or None to use all found
)

df_rmse_relu_correlated_no_lambda, df_posterior_rmse_relu_correlated_no_lambda = compute_rmse_from_fits(
    relu_fits_correlated_no_lambda, model_names_relu_no_lambda, folder = "friedman_correlated"
)

# Evaluate tanh models
df_rmse_tanh_no_lambda, df_posterior_rmse_tanh_no_lambda = compute_rmse_from_fits(
    tanh_fits_no_lambda, model_names_tanh_no_lambda
)

df_rmse_tanh_correlated_no_lambda, df_posterior_rmse_tanh_correlated_no_lambda = compute_rmse_from_fits(
    tanh_fits_correlated_no_lambda, model_names_tanh_no_lambda, folder = "friedman_correlated"
)


In [7]:
import pandas as pd
# NO LAMBDA
df1_no_lambda = df_rmse_relu_no_lambda.assign(activation="ReLU", setting="Original")
df2_no_lambda = df_rmse_tanh_no_lambda.assign(activation="Tanh", setting="Original")
df3_no_lambda = df_rmse_relu_correlated_no_lambda.assign(activation="ReLU", setting="Correlated")
df4_no_lambda = df_rmse_tanh_correlated_no_lambda.assign(activation="Tanh", setting="Correlated")

df_all_no_lambda = pd.concat([df1_no_lambda, df2_no_lambda, df3_no_lambda, df4_no_lambda], ignore_index=True)


df1_pm_no_lambda = df_posterior_rmse_relu_no_lambda.assign(activation="ReLU", setting="Original")
df2_pm_no_lambda = df_posterior_rmse_tanh_no_lambda.assign(activation="Tanh", setting="Original")
df3_pm_no_lambda = df_posterior_rmse_relu_correlated_no_lambda.assign(activation="ReLU", setting="Correlated")
df4_pm_no_lambda = df_posterior_rmse_tanh_correlated_no_lambda.assign(activation="Tanh", setting="Correlated")

df_all_pm_no_lambda = pd.concat([df1_pm_no_lambda, df2_pm_no_lambda, df3_pm_no_lambda, df4_pm_no_lambda], ignore_index=True)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# --- prepare data ---
df = df_all_no_lambda.copy()

abbr = {
    "Dirichlet Horseshoe": "DHS",
    "Dirichlet Student T": "DST",    
    "Beta Horseshoe": "BHS",
    "Beta Student T": "BST",
    "Dirichlet tau": r"Dir - $\tau$",
    "Beta tau": r"Beta - $\tau$",
    "Dirichlet": r"Dir",
    "Beta": r"Beta",
}

# unify model names across activations (strip " tanh")
df["model_clean"] = df["model"].str.replace(" tanh", "", regex=False)

# summary stats per (setting, N, model, activation)
summary = (
    df.groupby(["setting", "N", "model_clean", "activation"], as_index=False)["rmse"]
      .agg(mean="mean", std="std")
)

# plotting order
settings = ["Original", "Correlated"]
Ns = [50, 100, 200, 500]
models = ["Dirichlet tau", "Beta tau", "Dirichlet", "Beta"]

# visuals
markers = {"ReLU": "o", "Tanh": "^"}            # shapes
offsets = {"ReLU": -0.12, "Tanh": +0.12}        # side-by-side jitter on x
model_offsets = {
    "Dirichlet Horseshoe": -0.08,
    "Dirichlet Student T": -0.06, 
    "Beta Horseshoe": -0.04, 
    "Beta Student T": -0.02,
    "Dirichlet tau": +0.02,
    "Beta tau": +0.04,
    "Dirichlet": +0.06,
    "Beta": +0.08,
}
palette_list = plt.get_cmap("tab10").colors
palette = {m: palette_list[i] for i, m in enumerate(models)}

# map N to base x positions and add offsets for activation
xbase = {N: i for i, N in enumerate(Ns)}

fig, axes = plt.subplots(1, 2, figsize=(12, 7), sharey=True)

for ax, setting in zip(axes, settings):
    sub = summary[summary["setting"] == setting]
    # plot each model+activation with errorbars, without lines
    for m in models:
        for act in ["ReLU", "Tanh"]:
            g = sub[(sub["model_clean"] == m) & (sub["activation"] == act)]
            if g.empty:
                continue
            #xs = [xbase[n] + offsets[act] for n in g["N"]]
            xs = [xbase[n] + offsets[act] + model_offsets[m] for n in g["N"]]

            ax.errorbar(
                xs, g["mean"], yerr=g["std"],
                fmt=markers[act], markersize=10,
                linestyle="none", capsize=3,
                color=palette[m], markeredgecolor="black"
            )

    ax.set_title(f"{setting}", fontsize=15)
    ax.set_xticks(range(len(Ns)))
    ax.set_xticklabels(Ns, fontsize=15)
    ax.set_xlabel("N", fontsize=15)
    ax.set_ylabel("RMSE", fontsize=15)
    ax.tick_params(axis='y', labelsize=15)
    ax.grid()

# --- legends ---
model_handles = [
    Line2D(
        [0], [0],
        marker="o",
        linestyle="none",
        color=palette[m],
        markeredgecolor="black",
        markersize=12,
        label=abbr.get(m, m)   # <- use abbreviation
    )
    for m in models
]

# activation legend (shapes)
activation_handles = [
    Line2D([0], [0], marker=markers["ReLU"], linestyle="none", color="black",
           markersize=12, label="ReLU"),
    Line2D([0], [0], marker=markers["Tanh"], linestyle="none", color="black",
           markersize=12, label="Tanh"),
]

for ax in axes:
    ax.legend(
        handles=model_handles + activation_handles,
        title=None,
        loc="upper right",
        frameon=False,
        ncol=1,
        fontsize = 14
    )
plt.tight_layout(rect=(0, 0, 1, 1))
plt.grid()
#plt.savefig("figures_for_use_in_paper/friedman_RMSE_with_beta.pdf", bbox_inches="tight")
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
# --- prepare data ---
df = df_all_pm_no_lambda.copy()

# unify model names across activations (strip " tanh")
df["model_clean"] = df["model"].str.replace(" tanh", "", regex=False)

# summary stats per (setting, N, model, activation)
summary = (
    df.groupby(["setting", "N", "model_clean", "activation"], as_index=False)["posterior_mean_rmse"]
      .agg(mean="mean", std="std")
)
# plotting order
settings = ["Original", "Correlated"]
Ns = [50, 100, 200, 500]
models = ["Dirichlet tau", "Beta tau", "Dirichlet", "Beta"]

# visuals
markers = {"ReLU": "o", "Tanh": "^"}            # shapes
offsets = {"ReLU": -0.12, "Tanh": +0.12}        # side-by-side jitter on x
model_offsets = {
    "Dirichlet tau": -0.04,
    "Beta tau": -0.01,
    "Dirichlet": +0.01,
    "Beta": +0.04,
}
palette_list = plt.get_cmap("tab10").colors
palette = {m: palette_list[i] for i, m in enumerate(models)}

# map N to base x positions and add offsets for activation
xbase = {N: i for i, N in enumerate(Ns)}

fig, axes = plt.subplots(1, 2, figsize=(12, 5), sharey=True)

for ax, setting in zip(axes, settings):
    sub = summary[summary["setting"] == setting]
    # plot each model+activation with errorbars, without lines
    for m in models:
        for act in ["ReLU", "Tanh"]:
            g = sub[(sub["model_clean"] == m) & (sub["activation"] == act)]
            if g.empty:
                continue
            #xs = [xbase[n] + offsets[act] for n in g["N"]]
            xs = [xbase[n] + offsets[act] + model_offsets[m] for n in g["N"]]
            
            ax.plot(
                xs, g["mean"],
                marker=markers[act],
                markersize=7,
                linestyle="none",
                color=palette[m],
                markeredgecolor="black",
            )


    ax.set_title(f"{setting}")
    ax.set_xticks(range(len(Ns)))
    ax.set_xticklabels(Ns)
    ax.set_xlabel("N")
    ax.set_ylabel("RMSE")
    ax.grid()

# --- legends ---
model_handles = [
    Line2D(
        [0], [0],
        marker="o",
        linestyle="none",
        color=palette[m],
        markeredgecolor="black",
        markersize=7,
        label=abbr.get(m, m)   # <- use abbreviation
    )
    for m in models
]

# activation legend (shapes)
activation_handles = [
    Line2D([0], [0], marker=markers["ReLU"], linestyle="none", color="black",
           markersize=7, label="ReLU"),
    Line2D([0], [0], marker=markers["Tanh"], linestyle="none", color="black",
           markersize=7, label="Tanh"),
]

for ax in axes:
    ax.legend(
        handles=model_handles + activation_handles,
        title=None,
        loc="upper right",
        frameon=False,
        ncol=1
    )
plt.tight_layout(rect=(0, 0, 1, 1))
#plt.grid()
plt.show()

In [10]:
from utils.sparsity import forward_pass_relu, forward_pass_tanh, local_prune_weights

def compute_sparse_rmse_results(seeds, models, all_fits, get_N_sigma, forward_pass, folder,
                         sparsity=0.0, prune_fn=None):
    results = []
    posterior_means = []

    for seed in seeds:
        N, sigma = get_N_sigma(seed)
        dataset_key = f'Friedman_N{N}_p10_sigma{sigma:.2f}_seed{seed}'
        path = f"datasets/{folder}/{dataset_key}.npz"

        try:
            data = np.load(path)
            X_test, y_test = data["X_test"], data["y_test"]
        except FileNotFoundError:
            print(f"[SKIP] File not found: {path}")
            continue

        for model in models:
            try:
                fit = all_fits[dataset_key][model]['posterior']
                W1_samples = fit.stan_variable("W_1")           # (S, P, H)
                W2_samples = fit.stan_variable("W_L")           # (S, H, O)
                b1_samples = fit.stan_variable("hidden_bias")   # (S, O, H)
                b2_samples = fit.stan_variable("output_bias")   # (S, O)
            except KeyError:
                print(f"[SKIP] Model or posterior not found: {dataset_key} -> {model}")
                continue

            S = W1_samples.shape[0]
            rmses = np.zeros(S)
            #print(y_test.shape)
            y_hats = np.zeros((S, y_test.shape[0]))

            for i in range(S):
                W1 = W1_samples[i]
                W2 = W2_samples[i]

                # Apply pruning mask if requested
                if prune_fn is not None and sparsity > 0.0:
                    masks = prune_fn([W1, W2], sparsity)
                    W1 = W1 * masks[0]
                    #W2 = W2 * masks[1]

                y_hat = forward_pass(X_test, W1, b1_samples[i][0], W2, b2_samples[i])
                y_hats[i] = y_hat.squeeze()  # Store the prediction for each sample
                rmses[i] = np.sqrt(np.mean((y_hat.squeeze() - y_test)**2))
                
            posterior_mean = np.mean(y_hats, axis=0)
            posterior_mean_rmse = np.sqrt(np.mean((posterior_mean - y_test.squeeze())**2))

            posterior_means.append({
                'seed': seed,
                'N': N,
                'sigma': sigma,
                'model': model,
                'sparsity': sparsity,
                'posterior_mean_rmse': posterior_mean_rmse
            })

            for i in range(S):
                results.append({
                    'seed': seed,
                    'N': N,
                    'sigma': sigma,
                    'model': model,
                    'sparsity': sparsity,
                    'rmse': rmses[i]
                })

    df_rmse = pd.DataFrame(results)
    df_posterior_rmse = pd.DataFrame(posterior_means)

    return df_rmse, df_posterior_rmse


sparsity_levels = [0.0, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

seeds = [1, 2, 11]
seeds_correlated = [1, 6, 11]

def get_N_sigma(seed):
    if seed == 1:
        N=100
    elif seed == 2:
        N=200
    else:
        N=500
    sigma=1.00
    return N, sigma

def get_N_sigma_correlated(seed):
    if seed == 1:
        N=100
    elif seed == 6:
        N=200
    else:
        N=500
    sigma=1.00
    return N, sigma

In [ ]:
df_rmse_relu_no_lambda, df_posterior_rmse_relu_no_lambda = {}, {}
df_rmse_relu_correlated_no_lambda, df_posterior_rmse_relu_correlated_no_lambda = {}, {}
df_rmse_tanh_no_lambda, df_posterior_rmse_tanh_no_lambda = {}, {}
df_rmse_tanh_correlated_no_lambda, df_posterior_rmse_tanh_correlated_no_lambda = {}, {}

for sparsity in sparsity_levels:
    df_rmse_relu_no_lambda[sparsity], df_posterior_rmse_relu_no_lambda[sparsity] = compute_sparse_rmse_results(
        seeds, model_names_relu_no_lambda, relu_fits_no_lambda, get_N_sigma, forward_pass_relu, folder = "friedman",
        sparsity=sparsity, prune_fn=local_prune_weights
    )
    
    df_rmse_relu_correlated_no_lambda[sparsity], df_posterior_rmse_relu_correlated_no_lambda[sparsity] = compute_sparse_rmse_results(
        seeds_correlated, model_names_relu_no_lambda, relu_fits_correlated_no_lambda, get_N_sigma_correlated, forward_pass_relu, folder = "friedman_correlated",
        sparsity=sparsity, prune_fn=local_prune_weights
    )
    
    df_rmse_tanh_no_lambda[sparsity], df_posterior_rmse_tanh_no_lambda[sparsity] = compute_sparse_rmse_results(
        seeds, model_names_tanh_no_lambda, tanh_fits_no_lambda, get_N_sigma, forward_pass_tanh, folder = "friedman",
        sparsity=sparsity, prune_fn=local_prune_weights
    )
    
    df_rmse_tanh_correlated_no_lambda[sparsity], df_posterior_rmse_tanh_correlated_no_lambda[sparsity] = compute_sparse_rmse_results(
        seeds_correlated, model_names_tanh_no_lambda, tanh_fits_correlated_no_lambda, get_N_sigma_correlated, forward_pass_tanh, folder = "friedman_correlated",
        sparsity=sparsity, prune_fn=local_prune_weights
    )

In [12]:
import pandas as pd

df_rmse_full_relu_no_lambda = pd.concat(
    [df.assign(sparsity=sparsity) for sparsity, df in df_rmse_relu_no_lambda.items()],
    ignore_index=True
)

df_rmse_full_relu_correlated_no_lambda = pd.concat(
    [df.assign(sparsity=sparsity) for sparsity, df in df_rmse_relu_correlated_no_lambda.items()],
    ignore_index=True
)

df_rmse_full_tanh_no_lambda = pd.concat(
    [df.assign(sparsity=sparsity) for sparsity, df in df_rmse_tanh_no_lambda.items()],
    ignore_index=True
)

df_rmse_full_tanh_correlated_no_lambda = pd.concat(
    [df.assign(sparsity=sparsity) for sparsity, df in df_rmse_tanh_correlated_no_lambda.items()],
    ignore_index=True
)


In [25]:
df_tanh_o_no_lambda = df_rmse_full_tanh_no_lambda.copy()
df_tanh_o_no_lambda["model"] = df_tanh_o_no_lambda["model"].str.replace(" tanh", "", regex=False)

df_tanh_c_no_lambda= df_rmse_full_tanh_correlated_no_lambda.copy()
df_tanh_c_no_lambda["model"] = df_tanh_c_no_lambda["model"].str.replace(" tanh", "", regex=False)

df_relu_o_no_lambda = df_rmse_full_relu_no_lambda.copy()
df_relu_c_no_lambda = df_rmse_full_relu_correlated_no_lambda.copy()


In [27]:
abbr = {
    "Dirichlet tau": r"Dir - $\tau$",
    "Beta tau": r"Beta - $\tau$",
    "Dirichlet": r"Dir",
    "Beta": r"Beta",
}

palette_abbr = {
    r"Dir - $\tau$": "C1",
    r"Beta - $\tau$": "C2",
    r"Dir": "C3",
    r"Beta": "C4",
}

def plot_rmse_one_figure(
    df_all,
    Ns=(100, 200, 500), figsize=(12, 12), title="Original vs Correlated (tanh vs ReLU)"
):

    # Orderings
    setting_order = ["Original", "Correlated"]
    activation_order = ["tanh", "ReLU"]

    # Seaborn aesthetics (keeps your 'talk' sizing / whitegrid)
    #sns.set_context("talk")
    sns.set_style("whitegrid")
    plt.rcParams.update({
        "axes.spines.top": False,
        "axes.spines.right": False,
        "legend.frameon": True
    })

    fig, axes = plt.subplots(2, len(Ns), figsize=figsize, sharex=True, sharey=False)
    if len(Ns) == 1:
        axes = axes.reshape(2, 1)

    # We’ll plot using seaborn’s style mapping (style=activation, markers=True, dashes=True)
    # so tanh vs ReLU are visually distinct and consistent across the grid.
    for j, Nval in enumerate(Ns):
        for i, setting in enumerate(setting_order):
            ax = axes[i, j]
            dfN = df_all[(df_all["N"] == Nval) & (df_all["setting"] == setting)].copy()
            # Safety: if empty, skip
            if dfN.empty:
                ax.set_visible(False)
                continue

            # Use abbreviated labels on the legend (we’ll build custom legends later anyway)
            #dfN["model_abbr"] = dfN["model"].map(lambda m: abbr.get(m, m))
            dfN["model_abbr"] = dfN["model"].map(lambda m: abbr.get(m, m))

            sns.lineplot(
                data=dfN,
                x="sparsity",
                y="rmse",
                hue="model_abbr",
                style="activation",
                markers=True,
                dashes=True,
                palette=palette_abbr,
                hue_order=[abbr[m] for m in models if m in dfN["model"].unique()],
                style_order=activation_order,
                errorbar=None,
                ax=ax,
            )
            #ax.set_title(f"N={Nval}")
            ax.set_title(f"N={Nval}", fontweight="normal", fontsize=15)

            ax.set_xlabel("Sparsity", fontsize=15)
            ax.set_ylabel("RMSE" if j == 0 else "", fontsize=15)
            ax.tick_params(axis='both', labelsize=10)
            ax.grid(True, which="major", alpha=0.25)
            if ax.legend_:  # remove local legends
                ax.legend_.remove()

    # ---------- Build two clean, global legends ----------
    # 1) Prior legend (colors), using abbreviations in desired order present in data
    models_present = []
    for m in ["Dirichlet tau", "Beta tau", "Dirichlet", "Beta"]:
        if (df_all["model"] == m).any():
            models_present.append(m)
    prior_handles = [
        Line2D([0],[0], color=palette_abbr[abbr[m]], marker='o', linestyle='-', linewidth=2, markersize=12)
        for m in models_present
    ]
    prior_labels = [abbr[m] for m in models_present]

    # 2) Activation legend (styles) – black lines with linestyle/markers
    # Let seaborn pick the default mapping; we emulate a solid for tanh and dashed for ReLU.
    act_style = {
        "tanh": dict(linestyle='-'),#, marker='o'),
        "ReLU": dict(linestyle='--'),#, marker='s'),
    }
    activation_handles = [
        Line2D([0],[0], color='black', linewidth=2, markersize=12, **act_style[act])
        for act in activation_order
        if (df_all["activation"] == act).any()
    ]
    activation_labels = [act for act in activation_order if (df_all["activation"] == act).any()]

    # Place legends: priors on top center, activations below it
    # (Adjust bbox_to_anchor if you prefer side-by-side or bottom placement.)
    if prior_handles:
        leg1 = fig.legend(
            prior_handles, prior_labels,
            title="Prior",
            loc="upper right",
            ncol=len(prior_handles),
            frameon=True,
            bbox_to_anchor=(0.7, 1.02),
            fontsize = 15
        )
        fig.add_artist(leg1)
    if activation_handles:
        fig.legend(
            activation_handles, activation_labels,
            title="Activation",
            loc="upper left",
            ncol=len(activation_handles),
            frameon=True,
            bbox_to_anchor=(0.7, 1.02),
            fontsize = 15
        )
    #fig.suptitle(title, y=1.08, fontsize=18)
    plt.tight_layout(rect=[0, 0.4, 0.95, 0.95])
    #plt.savefig("figures_for_use_in_paper/friedman_sparsity_with_beta.pdf", bbox_inches="tight")
    plt.show()


In [28]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.lines import Line2D
import pandas as pd
from collections import OrderedDict

def make_merged_df(
    df_tanh_o, df_tanh_c, df_relu_o, df_relu_c,
    drop_tanh_suffix=True
):
    """Return one long df with columns: N, sparsity, rmse, model, activation, setting."""
    dfs = []
    # Tanh (optionally strip ' tanh' from model names if present)
    for df, setting in [(df_tanh_o, "Original"), (df_tanh_c, "Correlated")]:
        d = df.copy()
        if drop_tanh_suffix and " tanh" in "".join(d["model"].unique()):
            d["model"] = d["model"].str.replace(" tanh", "", regex=False)
        d["activation"] = "tanh"
        d["setting"] = setting
        dfs.append(d)
    # ReLU
    for df, setting in [(df_relu_o, "Original"), (df_relu_c, "Correlated")]:
        d = df.copy()
        d["activation"] = "ReLU"
        d["setting"] = setting
        dfs.append(d)
    out = pd.concat(dfs, ignore_index=True)

    # Keep only models that exist in BOTH activations so legend doesn't show ghosts
    models_tanh = set(out.loc[out.activation=="tanh","model"].unique())
    models_relu = set(out.loc[out.activation=="ReLU","model"].unique())
    common_models = sorted(list(models_tanh & models_relu))
    if common_models:
        out = out[out["model"].isin(common_models)]
    return out

df_all = make_merged_df(df_tanh_o_no_lambda, df_tanh_c_no_lambda, df_relu_o_no_lambda, df_relu_c_no_lambda)



In [ ]:
plot_rmse_one_figure(df_all,
                     Ns=(100, 200, 500),
                     title="Original vs Correlated")

#### ORIGINAL DATASET - ENDRE RUN_REGRESSION.py
python3 utils/run_all_regression_models.py --model beta_tau_tanh --output_dir results/regression/single_layer/relu/friedman/no_lambda &&
python3 utils/run_all_regression_models.py --model dirichlet_tau_tanh --output_dir results/regression/single_layer/relu/friedman/no_lambda &&

python3 utils/run_all_regression_models.py --model beta --output_dir results/regression/single_layer/relu/friedman/no_lambda &&
python3 utils/run_all_regression_models.py --model dirichlet --output_dir results/regression/single_layer/relu/friedman/no_lambda &&

python3 utils/run_all_regression_models.py --model beta_tanh --output_dir results/regression/single_layer/relu/friedman/no_lambda &&
python3 utils/run_all_regression_models.py --model dirichlet_tanh --output_dir results/regression/single_layer/relu/friedman/no_lambda

In [ ]:
import numpy as np
from scipy.stats import dirichlet, beta

np.random.seed(42)

K = 10
alpha = 0.1

# Dirichlet draw
dirichlet_draw = dirichlet.rvs(alpha=np.full(K, alpha), size=1)[0]

# Independent Beta draws (same marginals)
beta_draw = beta.rvs(alpha, (K - 1) * alpha, size=K)

print("Dirichlet draw:")
print(dirichlet_draw)
print("Sum:", dirichlet_draw.sum())

print("\nIndependent Beta draws:")
print(beta_draw)
print("Sum:", beta_draw.sum())


In [ ]:
N = 100_000

dirichlet_sums = np.ones(N)  # always 1
beta_sums = beta.rvs(alpha, (K - 1) * alpha, size=(N, K)).sum(axis=1)

print("Beta sum: mean =", beta_sums.mean())
print("Beta sum: std  =", beta_sums.std())
print("Beta sum: min/max =", beta_sums.min(), beta_sums.max())


In [ ]:
dirichlet_samples = dirichlet.rvs(
    alpha=np.full(K, alpha), size=N
)

beta_samples = beta.rvs(
    alpha, (K - 1) * alpha, size=(N, K)
)

print("Dirichlet off-diagonal correlation (approx):",
      np.corrcoef(dirichlet_samples.T)[0, 1])

print("Beta off-diagonal correlation (approx):",
      np.corrcoef(beta_samples.T)[0, 1])


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import dirichlet, beta

# --- Settings ---
np.random.seed(42)
K = 10
alpha = 0.1
N = 80_000  # Monte Carlo draws

# --- 1) Draw variance vectors ---
v_dir = dirichlet.rvs(alpha=np.full(K, alpha), size=N)            # sums to 1
v_beta = beta.rvs(alpha, (K - 1) * alpha, size=(N, K))            # iid, same marginals

# --- 2) Draw weights with those variances: w_i ~ N(0, v_i) ---
w1 = np.random.normal(loc=0.0, scale=v_dir, size=(N, K))   # Dirichlet variances
w2 = np.random.normal(loc=0.0, scale=v_beta, size=(N, K))  # Beta variances

# --- 3) Diagnostics for plotting ---
sum_dir = v_dir.sum(axis=1)      # == 1
sum_beta = v_beta.sum(axis=1)

max_dir = v_dir.max(axis=1)
max_beta = v_beta.max(axis=1)

l2_w1 = np.linalg.norm(w1, axis=1)
l2_w2 = np.linalg.norm(w2, axis=1)

maxabs_w1 = np.max(np.abs(w1), axis=1)
maxabs_w2 = np.max(np.abs(w2), axis=1)

# Correlation matrices (neutral scaling: always plot on [-1, 1])
corr_dir = np.corrcoef(v_dir.T)
corr_beta = np.corrcoef(v_beta.T)

# ------------------------------------------------------------
# PLOT A: Sum of variances (Dirichlet fixed at 1 vs Beta random)
# ------------------------------------------------------------
plt.figure()
plt.hist(sum_beta, bins=80, density=True, alpha=0.8,
         label='Sum of iid Beta variances')
plt.axvline(1.0, linewidth=2, label='Dirichlet sum (always 1)')
plt.xlim(0, K)
plt.xlabel('Sum of 10 variance components')
plt.ylabel('Density')
plt.title('Sum constraint: Dirichlet vs iid Betas (same marginals)')
plt.legend()
plt.show()

# ------------------------------------------------------------
# PLOT B: Joint behavior (v1 vs v2) scatter with fixed axes [0,1]
# ------------------------------------------------------------
m = 6000
idx = np.random.choice(N, size=m, replace=False)

plt.figure()
plt.scatter(v_dir[idx, 0], v_dir[idx, 1], s=10, alpha=0.3,
            label='Dirichlet (v1 vs v2)')
plt.scatter(v_beta[idx, 0], v_beta[idx, 1], s=10, alpha=0.3,
            label='iid Betas (v1 vs v2)')
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.xlabel('v1')
plt.ylabel('v2')
plt.title('Two components (fixed [0,1] axes)')
plt.legend()
plt.show()

# ------------------------------------------------------------
# PLOT C: Correlation heatmap (Dirichlet) with neutral fixed scale
# ------------------------------------------------------------
plt.figure()
plt.imshow(corr_dir, vmin=-1, vmax=1, interpolation='nearest')
plt.colorbar(label='Correlation')
plt.title('Corr(variance components): Dirichlet (fixed scale [-1,1])')
plt.xticks(range(K))
plt.yticks(range(K))
plt.show()

# ------------------------------------------------------------
# PLOT D: Correlation heatmap (iid Betas) with neutral fixed scale
# ------------------------------------------------------------
plt.figure()
plt.imshow(corr_beta, vmin=-1, vmax=1, interpolation='nearest')
plt.colorbar(label='Correlation')
plt.title('Corr(variance components): iid Betas (fixed scale [-1,1])')
plt.xticks(range(K))
plt.yticks(range(K))
plt.show()

# ------------------------------------------------------------
# PLOT E: Induced weight scale comparison via ||w||_2
# ------------------------------------------------------------
xmax = float(np.quantile(np.concatenate([l2_w1, l2_w2]), 0.995))

plt.figure()
plt.hist(l2_w1, bins=120, density=True, alpha=0.6,
         label=r'$\|w_1\|_2$,  $v\sim$ Dirichlet,  $w|v\sim N(0,\mathrm{diag}(v))$')
plt.hist(l2_w2, bins=120, density=True, alpha=0.6,
         label=r'$\|w_2\|_2$,  $v_i\sim$ iid Beta,  $w|v\sim N(0,\mathrm{diag}(v))$')
plt.xlim(0, xmax)
plt.xlabel(r'$\|w\|_2$')
plt.ylabel('Density')
plt.title('Weights induced by variance vectors: Dirichlet vs iid Betas')
plt.legend()
plt.show()

# (Optional) PLOT F: Induced max |w_i|
xmax2 = float(np.quantile(np.concatenate([maxabs_w1, maxabs_w2]), 0.995))

plt.figure()
plt.hist(maxabs_w1, bins=120, density=True, alpha=0.6, label=r'$\max_i |w_{1,i}|$')
plt.hist(maxabs_w2, bins=120, density=True, alpha=0.6, label=r'$\max_i |w_{2,i}|$')
plt.xlim(0, xmax2)
plt.xlabel(r'$\max_i |w_i|$')
plt.ylabel('Density')
plt.title('Induced extreme weights: Dirichlet vs iid Betas')
plt.legend()
plt.show()
